In [1]:
import requests
import pandas as pd
import numpy as np
import jieba
import zlib
import re
import jieba.posseg as pseg
from sklearn.feature_extraction.text import CountVectorizer  

In [2]:
jieba.load_userdict("C:\\Users\\12405\\Desktop\\danmu\\word_jieba.txt")

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\12405\AppData\Local\Temp\jieba.cache
Loading model cost 0.811 seconds.
Prefix dict has been built successfully.


In [3]:
tv_ids=[13661233000,13847100500,13950421000,13958052300,14140651000,14139369700,14219902900,14224268700,14425793300,
        14426429300,14520762300,14520454600,14722962600,14722978600,14812202100,148122307,14992485500,14992525500,15068699100,
        15068748500,15235939800,1523742100,15303652400,15305002700,15472234400,15472264800,15535092800,15535228800,15687653300,15687669100,
        15747448600,15746646600,15893731300,15894809200,15958321100,15958332600,16099208800,16097589000,16152560100,16153619000]
dict_tvid={}
idd=1
for id in tv_ids:
    tv_id=str(id);
    for page in range(1, 2):#这里就取每一个文档的第一个弹幕包进行解压，来构建候选情感词典
        # https://cmts.iqiyi.com/bullet/tv_id[-4:-2]/tv_id[-2:]/tv_id_300_x.z
        # https://cmts.iqiyi.com/bullet/视频编号的倒数4、3位/视频编号的倒数2、1位/视频编号_300_序号.z
        # 弹幕文件每5分钟（300秒）向服务器请求一次，故每集弹幕文件数量等于视频时间除以300之后向上取整，实际编程时这里可以简单处理
            url = 'https://cmts.iqiyi.com/bullet/'\
                + tv_id[-4:-2] + '/'\
                + tv_id[-2:] + '/'\
                + tv_id + '_300_'\
                + str(page) + '.z'
        # 请求弹幕压缩文件
            res = requests.get(url).content
            res_byte = bytearray(res)
            try:
                xml = zlib.decompress(res_byte).decode('utf-8')
                comments=re.findall("<content>(.*?)</content>",xml)#获取弹幕
                ids=re.findall("<contentId>(.*?)</contentId",xml)
                data=pd.DataFrame({"comment":comments,
                                  "ID":ids})
      
                score=[];    #保存每条弹幕的情感分数   这里是用SNOWNLP来完成的打分，效果不是特别好
                for comment in data['comment'].tolist():
                    try:
                        s=SnowNLP(comment[0])
                        score.append(s.sentiments)
                    except:
                        print("something is wrong")
                        score.append(0.5)
                #区间转换成-0.5—0.5，更能直观判断感情
                result=[]
                i=0;
                while i<len(score):
                    result.append(score[i]-0.5)
                    i=i+1
                data['score']=result
                dict_tvid[idd]=data
            except:
                print('异常')
    idd=idd+1

异常
异常


In [4]:
dict_tvid

{1:                        ID                  comment     score
 0     1584014025800000334                    莎莎我来啦  0.000000
 1     1584592234604004775                    帅啊！坤坤  0.433333
 2     1584016078119007102                    夏研我来啦  0.326087
 3     1584014107955006164                  虞书欣妈妈爱你  0.000000
 4     1584014053937000878                 来了来了，蔡PD  0.026233
 5     1585797248204002870              蛋黄的长裙，蓬松的头发 -0.060976
 6     1584014127356001221         来啦来啦，虞书欣小姐姐我来看你啦  0.026233
 7     1584068531938004733                春暖花开，为坤而来  0.119048
 8     1584013992665009747                提前了啊，不错不错 -0.060510
 9     1584014089369007414                   刚刚走错门了 -0.001479
 10    1584014578717009276                   女神们集合了  0.104938
 11    1584014060998005128              太快了太快了23333 -0.156125
 12    1584015896782009047               啊啊啊阿坤坤我来了！  0.026233
 13    1584013998388005554                啊啊啊崽崽妈妈爱你  0.026233
 14    1584016978189007983                 期待了一个多月了  0.088235
 15  

In [137]:
import codecs

得到基础情感词典
1.用TF-IDF训练，得到带权重的候选词，然后找出高权重候选词
2.用word2vec训练，得到候选词的词向量
3.提取候选情感词：得到word2vec结果中高权重的词
4.再加入基础词集

In [152]:
#现在我们自己来构建情感词典
#BosonNLP是基于微博、新闻、论坛等数据来源构建的情感词典  BosonNLP_sentiment_score.txt
#snownlp也可以训练自己的模型
#候选词典
words_houxuan=[]
result = codecs.open("C:\\Users\\12405\\Desktop\\danmu\\带词性的分词结果.txt", 'w', 'utf-8')
for id in dict_tvid:    
    for comment in dict_tvid[id]["comment"]:
        for x in pseg.cut(comment):
    
            if x.flag.startswith('a') or x.flag.startswith('v'):#形容词和动词
                words_houxuan.append(x.word)
                line = str(x.word) + " "
                result.write(line)
#words_houxuan为得到的候选词集      


In [158]:
words=[line.strip().split(" ")for line in open("C:\\Users\\12405\\Desktop\\danmu\\带词性的分词结果.txt",encoding='utf-8').readlines()]

In [161]:
maxLen=100
num_featrues=100
min_word_count=3
num_workers=4
context=4
model=word2vec.Word2Vec(words,workers=num_workers,size=num_featrues,min_count=min_word_count,window=context)

2020-05-17 22:23:10,629: INFO: collecting all words and their counts
2020-05-17 22:23:10,632: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-05-17 22:23:10,670: INFO: collected 3974 word types from a corpus of 91669 raw words and 1 sentences
2020-05-17 22:23:10,672: INFO: Loading a fresh vocabulary
2020-05-17 22:23:10,688: INFO: effective_min_count=3 retains 1559 unique words (39% of original 3974, drops 2415)
2020-05-17 22:23:10,689: INFO: effective_min_count=3 leaves 88684 word corpus (96% of original 91669, drops 2985)
2020-05-17 22:23:10,715: INFO: deleting the raw counts dictionary of 3974 items
2020-05-17 22:23:10,718: INFO: sample=0.001 downsamples 70 most-common words
2020-05-17 22:23:10,720: INFO: downsampling leaves estimated 50094 word corpus (56.5% of prior 88684)
2020-05-17 22:23:10,731: INFO: estimated required memory for 1559 words and 100 dimensions: 2026700 bytes
2020-05-17 22:23:10,733: INFO: resetting layer weights
2020-05-17 22:23:11,33

In [163]:
model.wv.vocab

{'好': <gensim.models.keyedvectors.Vocab at 0x267e1fd56d8>,
 '可爱': <gensim.models.keyedvectors.Vocab at 0x267e1fd55f8>,
 '想': <gensim.models.keyedvectors.Vocab at 0x267e2dcb128>,
 '看': <gensim.models.keyedvectors.Vocab at 0x267e2dcb358>,
 '坐在': <gensim.models.keyedvectors.Vocab at 0x267e2dcb240>,
 '爱': <gensim.models.keyedvectors.Vocab at 0x267e2dcb208>,
 '欣欣': <gensim.models.keyedvectors.Vocab at 0x267e2dcb278>,
 '是': <gensim.models.keyedvectors.Vocab at 0x267e2dcb320>,
 '吃': <gensim.models.keyedvectors.Vocab at 0x267e2dcb1d0>,
 '少': <gensim.models.keyedvectors.Vocab at 0x267e2dcb0f0>,
 '出去': <gensim.models.keyedvectors.Vocab at 0x267e2dcb198>,
 '好吃': <gensim.models.keyedvectors.Vocab at 0x267e2dcb160>,
 '去': <gensim.models.keyedvectors.Vocab at 0x267e2dcb2e8>,
 '大': <gensim.models.keyedvectors.Vocab at 0x267e2dcb0b8>,
 '饿': <gensim.models.keyedvectors.Vocab at 0x267e2dcb2b0>,
 '可': <gensim.models.keyedvectors.Vocab at 0x267e2dcb390>,
 '对待': <gensim.models.keyedvectors.Vocab at 0x267e2

In [139]:
#候选词典基础词典求交集(找出候选词典中已经在基础词典中有分类的词)
stwlist = [line.strip() for line in open ('C:\\Users\\12405\\Desktop\\danmu\\BosonNLP_sentiment_score.txt',encoding='utf-8').readlines()]
#BosonNLP是基于微博、新闻、论坛等数据来源构建的情感词典  BosonNLP_sentiment_score.txt
words=[];
scores_words={};#保存在基础词典中每个词的评分
#读取到的是词和分数组合成的字符，中间用空格隔开 所以我们要进行分别保存
for x in stwlist:
    flag=0;
    for word in x.split(' '):
        if flag==0:
            words.append(word)
            
        if flag==1:
            scores_words[words[-1]]=word
            flag =0;
        flag = flag+1
#对候选词通过word2vec进行训练
#https://blog.csdn.net/weixin_40292043/article/details/79571346  这里面讲了很多参数 有时间可以一一调试 选择最合适的参数
     

1、取候选词集和基础词典的交集word_basic
基于word_basic计算种子情感词
2、得到word_basic之后，计算每个词的tf-idf值
3、用word2vec得到每个词的词向量
4、OW(i)=a*tfidfi+(1-a)*1/(n-1)*求和[sim(wordi,wordj)]
5.筛选OW值最大的前sn个词，即为种子情感词
词w在文档d中的词频tf (Term Frequency)，即词w在文档d中出现次数count(w, d)和文档d中总词数size(d)的比值：
·tf(w,d) = count(w, d) / size(d)
词w在整个文档集合中的逆向文档频率idf (Inverse Document Frequency)，即文档总数n与词w所出现文件数docs(w, D)比值的对数:
·idf = log(n / docs(w, D))
tf-idf模型根据tf和idf为每一个文档d和由关键词w[1]…w[k]组成的查询串q计算一个权值，用于表示查询串q与文档d的匹配度：
·tf-idf(q, d) = sum { i = 1..k | tf-idf(w[i], d) } = sum { i = 1..k | tf(w[i], d) * idf(w[i]) }

In [7]:
word_basic=[x for x in words_houxuan if x in words] #从有107303个词的候选词集以及114767个词的基础词集中得到包含94937个词的词集


In [164]:
model.save("C:\\Users\\12405\\Desktop\\danmu\\danmu_train_word.model")


2020-05-17 22:25:42,225: INFO: saving Word2Vec object under C:\Users\12405\Desktop\danmu\danmu_train_word.model, separately None
2020-05-17 22:25:42,228: INFO: not storing attribute vectors_norm
2020-05-17 22:25:42,230: INFO: not storing attribute cum_table
2020-05-17 22:25:42,255: INFO: saved C:\Users\12405\Desktop\danmu\danmu_train_word.model


In [165]:
model.wv.vocab

{'好': <gensim.models.keyedvectors.Vocab at 0x267e1fd56d8>,
 '可爱': <gensim.models.keyedvectors.Vocab at 0x267e1fd55f8>,
 '想': <gensim.models.keyedvectors.Vocab at 0x267e2dcb128>,
 '看': <gensim.models.keyedvectors.Vocab at 0x267e2dcb358>,
 '坐在': <gensim.models.keyedvectors.Vocab at 0x267e2dcb240>,
 '爱': <gensim.models.keyedvectors.Vocab at 0x267e2dcb208>,
 '欣欣': <gensim.models.keyedvectors.Vocab at 0x267e2dcb278>,
 '是': <gensim.models.keyedvectors.Vocab at 0x267e2dcb320>,
 '吃': <gensim.models.keyedvectors.Vocab at 0x267e2dcb1d0>,
 '少': <gensim.models.keyedvectors.Vocab at 0x267e2dcb0f0>,
 '出去': <gensim.models.keyedvectors.Vocab at 0x267e2dcb198>,
 '好吃': <gensim.models.keyedvectors.Vocab at 0x267e2dcb160>,
 '去': <gensim.models.keyedvectors.Vocab at 0x267e2dcb2e8>,
 '大': <gensim.models.keyedvectors.Vocab at 0x267e2dcb0b8>,
 '饿': <gensim.models.keyedvectors.Vocab at 0x267e2dcb2b0>,
 '可': <gensim.models.keyedvectors.Vocab at 0x267e2dcb390>,
 '对待': <gensim.models.keyedvectors.Vocab at 0x267e2

In [166]:
score_basic={}
for word in model.wv.vocab:
    
    if word in scores_words:
        score_basic[word]=float(scores_words[word])
print(len(model.wv.vocab))

1559


In [167]:
scores_order=sorted(score_basic.items(),key=lambda x:x[1],reverse=True)

In [168]:
#选取情感值最高的10个词作为正向词种子词，选取情感值最低的10个词作为负向词种子词
#这里依据基础情感词典中的评分选取种子词
 
i=0;
positive_words=[]
negative_words=[]
while(i<10):
    q=-(i+1)
    positive_words.append(scores_order[i])
    negative_words.append(scores_order[q])
    i=i+1;
print('positive',positive_words)
print('negative',negative_words)
##这里的种子词选取没有考虑tf-idf的值，只是基于在基础情感词典中的评分，所以这是一个可以优化的地方

positive [('甜美', 4.48307980713), ('选', 4.43746666368), ('抢眼', 3.99592290301), ('超可', 3.37612173454), ('搭配', 3.30237554184), ('捧场', 3.27228555549), ('闪耀', 3.26483471094), ('精致', 3.26234908657), ('支持', 3.24540391651), ('独特', 3.14351268948)]
negative [('救救', -4.33777722449), ('救命', -4.16534562268), ('该死', -3.85254830901), ('难受', -3.77980013251), ('烦', -3.55065786432), ('没完', -3.25977109288), ('饿', -3.00607419422), ('劳资', -2.99496261049), ('疼', -2.90732815497), ('肿', -2.88343090242)]


采用余弦距离来计算各候选情感词与种子词的相似度，
因为word_basic中每个词本身就是基于上下文语义生成的词向量， 因此，余弦值越大，则表明两个词语的语义越相似
分别计算每个候选词和种子词的情感相似度，再计算各余弦相似度和各个种子词的情感值的乘积，与每个候选情感词原有的情感值相加取平均值。
这里候选情感词的原有情感值是在情感词汇本体中找到的，如果没有在情感词汇本体中找到，那么这个候选情感词的初始情感值设为０
然后将各候选词对应每个种子词算出的情感值进行
求和取平均值 的 运 算，得到该候选情感词的情感值
若所得出的 ｗｏｒｄｓｃｏｒｅ＞０，则说明该候选词为正向情
感词，反之则为负向情感词    

In [169]:
final_word_score={}
for word in score_basic:
    word_score=0
    for word_positive in positive_words:
        word_similarity=model.similarity(word,word_positive[0]) #得到候选词和种子词的情感相似度
        word_score=word_score+(word_similarity*word_positive[1]+score_basic[word])/2#计算相似度和种子词情感值的乘积，与原有情感值相加再取平均
    for word_negative in negative_words:
        word_similarity=model.similarity(word,word_negative[0]) #得到候选词和种子词的情感相似度
        word_score=word_score+(word_similarity*word_negative[1]+score_basic[word])/2#计算相似度和种子词情感值的乘积，与原有情感值相加再取平均
    word_score=word_score/(len(positive_words)+len(negative_words))
    final_word_score[word]=word_score


C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """
C:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  


In [170]:
final_word_score#最终构建的情感词典

{'好': 0.9820136881469358,
 '可爱': 1.2195098690723867,
 '想': -0.5057831187789371,
 '看': -0.6613850883702865,
 '爱': 0.8137035591811689,
 '欣欣': 0.8526711129524662,
 '是': -0.18101679595464276,
 '吃': -0.1427680683641555,
 '少': -0.050426962409032924,
 '出去': -0.3116328562559995,
 '好吃': 0.679830734458333,
 '去': -0.216951007034517,
 '大': -0.0045890567715605205,
 '饿': -1.5622026687181028,
 '可': -0.01083179054145372,
 '对待': 0.11491622330025075,
 '要': -0.38707875227086264,
 '坐': -0.48424946963708126,
 '最好': 0.23399177607362298,
 '会': -0.5118970634038275,
 '长': -0.05593646022553844,
 '淘汰': -0.3414039221555827,
 '看看': 0.38210736106208043,
 '认识': 0.5547668140525107,
 '害': -0.545135645408549,
 '有钱': -0.3531906694305579,
 '完': -0.29534092556682967,
 '一般': 0.026386172666149517,
 '瘦': 0.42117012176239044,
 '想起': -0.19670292197972442,
 '开心': 1.2515882725432443,
 '好看': 0.5762218325755173,
 '有': 0.033976094725611014,
 '喝': -0.22091310874116316,
 '加油': 1.4327629093710135,
 '高': 0.6134196009415361,
 '看到': -0.0

In [171]:
final_word_scores=pd.DataFrame({"word":list(final_word_score.keys()),
                               "score":list(final_word_score.values())})
final_word_scores.to_csv('C:\\Users\\12405\\Desktop\\danmu\\feeling_dictionary.csv',encoding='utf-8',index=False)
#在有中文的时候，dataframe转换成csv文件会自动保存gdk文件，而读取会自动为utf-8格式  因此设置为utf-8，读取的时候就可以正常读取了
#为了防止读取的时候多出一列，这里设置index=False  不加索引

In [172]:
final_word_scores

,score,word
0,0.982014,好
1,1.219510,可爱
2,-0.505783,想
3,-0.661385,看
4,0.813704,爱
5,0.852671,欣欣
6,-0.181017,是
7,-0.142768,吃
8,-0.050427,少
9,-0.311633,出去
